<a href="https://colab.research.google.com/github/rdkdaniel/The-Swahili-Project/blob/main/The_Tokenizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Tokenizing the data**

In [ ]:
#https://towardsdatascience.com/designing-tokenizers-for-low-resource-languages-7faa4ab30ef4

**The data**

In [1]:
#Libraries
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('train.csv')

In [3]:
df.head()

,id,content,category
0,SW4670,"Bodi ya Utalii Tanzania (TTB) imesema, itafan...",uchumi
1,SW30826,"PENDO FUNDISHA-MBEYA RAIS Dk. John Magufuri, ...",kitaifa
2,SW29725,Mwandishi Wetu -Singida BENKI ya NMB imetoa ms...,uchumi
3,SW20901,"TIMU ya taifa ya Tanzania, Serengeti Boys jan...",michezo
4,SW12560,Na AGATHA CHARLES – DAR ES SALAAM ALIYEKUWA K...,kitaifa


In [4]:
df.describe()

,id,content,category
count,23268,23268,23268
unique,23268,22207,6
top,SW4670,"Na Mwandishi wetu, Mbeya Mamlaka ya Mapato Tan...",kitaifa
freq,1,86,10242


# **The Tokenizer**